[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openspyrit/spyrit-examples/blob/master/tutorial/tuto_train_split_meas_colab.ipynb)

# Tutorial to train a reconstruction network 

Tutorial to train a reconstruction network for 2D single-pixel imaging on stl10, for split measurements.

Training is performed by a call to *train.py*. Several parameters allow to modify acquisition, network and training (network architecture), optimisation and the use of tensorboard. 

Currently you can train the following networks by modifying the network architecture variable *arch*: 

- 'dc-net': Denoised Completion Network (DCNet). 
- 'pinv-net': Pseudo Inverse Network (PinvNet).
- 'upgd': Unrolled proximal gradient descent (UPGD). 

and the denoising variable *denoi*: 
- 'cnn': CNN no batch normalization
- 'cnnbn': CNN with batch normalization
- 'unet': UNet (0.5 M trainable parameters) 

## Settings and requirements

### Set google colab

On colab, choose GPU at *Runtime/Change runtime type*

In [ ]:
!nvidia-smi

### Dependencies

In [ ]:
import os
import datetime

First, mount google drive to import modules spyrit modules.

In [ ]:
mode_colab = True
if (mode_colab is True):
    # Connect to googledrive
    #if 'google.colab' in str(get_ipython()):
    # Mount google drive to access files via colab
    from google.colab import drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/MyDrive/

    # For the profiler
    !pip install -U tensorboard-plugin-profile

    # Load the TensorBoard notebook extension
    %load_ext tensorboard

### Clone Spyrit package

Clone and install spyrit and spyrit-examples respositories if not installed. Temporally, checkout to current development branch. 

In [ ]:
if (mode_colab is True):
    # Clone and install
    !git clone https://github.com/openspyrit/spyrit.git
    %cd spyrit
    !pip install -e .

    # Checkout to ongoing branch
    !git fetch --all

    # Add paths for modules
    import sys
    sys.path.append('./spyrit/core')
    sys.path.append('./spyrit/misc')
    sys.path.append('./spyrit/tutorial')
    %cd ..

    # Clone Spyrit-examples and checkout to branch tutorials
    !git clone https://github.com/openspyrit/spyrit-examples.git
    %cd spyrit-examples
    !git checkout train_upgd
    %cd tutorial

## Download data

Download covariance matrix. Alternatively install *openspyrit/spas* package:
```
    spyrit
    ├───stat
    │   ├───Average_64x64.npy
    │   ├───Cov_64x64.npy
    ├───spirit
```

In [ ]:
download_cov = True
if (download_cov is True):
    if 'stat' not in os.listdir():
        !pip install girder-client
        import girder_client

        # api Rest url of the warehouse
        url='https://pilot-warehouse.creatis.insa-lyon.fr/api/v1'
        
        # Generate the warehouse client
        gc = girder_client.GirderClient(apiUrl=url)

        #%% Download the covariance matrix and mean image
        data_folder = './stat/'
        dataId_list = [
                '63935b624d15dd536f0484a5', # for reconstruction (imageNet, 64)
                '63935a224d15dd536f048496', # for reconstruction (imageNet, 64)
                ]
        for dataId in dataId_list:
            myfile = gc.getFile(dataId)
            gc.downloadFile(dataId, data_folder + myfile['name'])

        print(f'Created {data_folder}') 
        !ls $data_folder

## Train

You can choose the following parameters:

- Measurement operators:
    - `meas`: Measurement type: 'hadam-split', 'hadam-pos'
    - `noise` = 'Noise types: 'poisson', 'gauss-approx', 'no-noise'
    - `prep` = Preprocessing types: 'dir-poisson', 'split-poisson'

- Acquisition:
    - `img_size`: Height / width dimension, default=64
    - `M`: Number of undersampling patterns, default=512
    - `subs`: Among 'var','rect', default="var"
    
- Network and training:
    - `data`: stl10 or imagenet, default="stl10"
    - `model_root`: Path to model saving files, default='./model/'
    - `data_root`: Path to the dataset, default="./data/"

    - `N0`: Mean maximum total number of photons, default=10
    - `stat_root`: Path to precomputed data, default="./stat/"
    - `arch`: Choose among 'dc-net','pinv-net', 'upgd', default="dc-net"
    - `denoi`: Choose among 'cnn','cnnbn', 'unet', default="unet"

- Specific models parameters
    - `upgd_iter`: Number of unrolled iterations for UPGD, default=6
    - `upgd_lamb`: Initial step size parameters for UPGD, default=1e-5


- Optimisation:
    - `num_epochs`: Number of training epochs, default=30
    - `batch_size`: Size of each training batch, default=512
    - `reg`: Regularisation Parameter, default=1e-7
    - `step_size`: Scheduler Step Size, default=10
    - `gamma`: Scheduler Decrease Rate, default=0.5
    - `checkpoint_model`: Optional path to checkpoint model, default=""
    - `checkpoint_interval`: Interval between saving model checkpoints, default=0
    - Training is done with *Adam* optimizer, *MSELoss*

- Tensorboard:
    - `tb_path`: Relative path for Tensorboard experiment tracking logs, default=False
    - `tb_prof`: Code profiler with Tensorboard, default=False
    - Logging of scalars *train_loss*, *val_loss* and images (dataset example ground-truth and predictions at different epochs).


In this tutorial, data is perturbed by Poisson noise (100 mean photons) and undersampling factor of 4, on stl10 dataset, and training is done with default parameters and using experiment tracking with tensorboard. 

In [ ]:
# Parameters
meas = 'hadam-split'    # measurement type
noise = 'poisson' # noise type
prep = 'split-poisson'    # preprocessing type
#
N0 = 100        # ph/pixel max: number of counts
img_size = 64   # image size
M =  img_size**2 // 4  # Num measurements = subsampled by factor 4
#
data_root = './data/'
data = 'stl10'
stat_root = './stat'
subs = 'var' # subsampling types: 'var': high variance, 'rect': low frequency
#
arch = 'dc-net' # Network architecture
denoi = 'unet' # Denoiser architecture
num_epochs = 30
batch_size = 128

# Tensorboard logs path
name_run = f"stl10_splitmeas_{subs}_{M}_{int(N0)}_{img_size}x{img_size}_{arch}_{denoi}"
mode_tb = True
if (mode_tb is True):
    if not os.path.exists(f'runs'):
        os.mkdir(f'runs')
    now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    tb_path = f'runs/runs_{name_run}/{now}'
    print(f"Tensorboard logdir {tb_path}")
else:
    tb_path = None
    
tb_prof = False # False
checkpoint_interval = 5

Training time: 2 min to download stl10, 2 min per epoch

In [ ]:
# Run train.py
if (mode_colab is True):
    !python3 train_gen_meas.py --meas $meas --noise $noise --prep $prep --N0 $N0 --M $M --data_root $data_root --data $data --stat_root $stat_root --subs $subs --tb_path $tb_path --tb_prof $tb_prof --arch $arch --denoi $denoi --num_epochs $num_epochs --img_size $img_size --checkpoint_interval $checkpoint_interval --batch_size $batch_size
else:
    import subprocess
    subprocess.run(['python3', 'train_gen_meas.py', '--meas', meas, '--noise', noise, '--prep', prep,
                '--data_root', data_root, '--data', data, '--stat_root', stat_root,
                '--N0', str(N0), '--M', str(M), '--subs', subs, '--img_size', str(img_size),
                '--arch', arch, '--denoi', denoi, '--num_epochs', str(num_epochs),
                #'--upgd_iter', str(upgd_iter),
                '--tb_path', tb_path,
                '--checkpoint_interval', str(checkpoint_interval, '--batch_size', str(batch_size))])

## Evaluate the trained model

### Tensorboard

Launch tensorboard. Select *SCALARS* or *IMAGES*. More options are available in the top-right corner.

In [ ]:
# Launch TensorBoard
# %tensorboard --logdir $tb_path
%tensorboard --logdir runs

In [ ]:
# If run twice tensorboard
#!lsof -i:6006
#!kill -9 17387